In [1]:
import pandas as pd
all_jobs = pd.read_csv("all_jobs.csv")

C:\Users\Admin\AppData\Local\Temp\ipykernel_21444\1021965397.py:2: DtypeWarning: Columns (1,6,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  all_jobs = pd.read_csv("all_jobs.csv")


In [ ]:
df = all_jobs.copy()
df.shape, df.columns

In [3]:
df.isna().mean().sort_values(ascending=False)

responsibilities    0.445337
qualifications      0.445337
country             0.445337
skills_raw          0.001042
company             0.000037
job_key             0.000034
location            0.000007
source              0.000000
job_title           0.000000
job_description     0.000000
dtype: float64

1-clean all jobs dataset

In [6]:
import pandas as pd
import re
from pathlib import Path


input_path  = "all_jobs.csv"               # merged big file (6GB)
output_path = "all_jobs_clean_full.csv"    # final cleaned file

chunksize = 50_000

# 🔐 for GLOBAL duplicate removal
seen_job_keys = set()       # for drop_duplicates(subset=["job_key"])
seen_composite_keys = set() # for drop_duplicates subset combo


# 4️⃣ your cleaning function, vectorized-style
def clean_text_series(s: pd.Series):
    return (
        s.astype(str)
         .str.replace("\n", " ", regex=False)
         .str.replace("\r", " ", regex=False)
         .str.replace(r"\s+", " ", regex=True)
         .str.strip()
    )

# 3️⃣ same logic: empty row = no title, no desc, no skills
def drop_fully_empty_rows(df: pd.DataFrame) -> pd.DataFrame:
    def is_empty(x):
        return (pd.isna(x)) or (str(x).strip() == "")

    mask_keep = ~(
        df["job_title"].apply(is_empty) &
        df["job_description"].apply(is_empty) &
        df["skills_raw"].apply(is_empty)
    )
    return df[mask_keep].copy()


def make_composite_key(row):
    # like your subset=["job_title","company","location","job_description"]
    parts = [
        str(row.get("job_title", "")).lower(),
        str(row.get("company", "")).lower(),
        str(row.get("location", "")).lower(),
        str(row.get("job_description", "")).lower()[:200],  # truncated desc to reduce memory
    ]
    return "||".join(parts)


def process_chunk(chunk: pd.DataFrame, first: bool):
    global seen_job_keys, seen_composite_keys

    # drop useless very sparse columns if they still exist
    for col in ["responsibilities", "qualifications","country"]:
        if col in chunk.columns:
            chunk = chunk.drop(columns=[col])

    # basic missing handling for text columns
    for col in ["job_title", "job_description", "skills_raw"]:
        if col in chunk.columns:
            chunk[col] = chunk[col].fillna("")

    # 👉 step 3: drop rows with empty title+desc+skills
    chunk = drop_fully_empty_rows(chunk)

    # 👉 step 4: clean text (job_title, job_description, skills_raw, location, company, country)
    cols_to_clean = ["job_title", "job_description", "skills_raw",
                     "location", "company", "country"]
    for col in cols_to_clean:
        if col in chunk.columns:
            chunk[col] = clean_text_series(chunk[col])

    # 👉 step 5: filter short title / desc
    if "job_title" in chunk.columns:
        chunk = chunk[chunk["job_title"].str.len() >= 5]
    if "job_description" in chunk.columns:
        chunk = chunk[chunk["job_description"].str.len() >= 30]

    # 👉 step 6.1: global dedup by job_key (if exists)
    if "job_key" in chunk.columns:
        mask_new_key = ~chunk["job_key"].isin(seen_job_keys)
        chunk = chunk[mask_new_key].copy()
        seen_job_keys.update(chunk["job_key"].tolist())

    # 👉 step 6.2: global dedup by combo (title+company+location+desc)
    #     (does what your drop_duplicates(subset=[...]) did, but globally)
    chunk["__dedup_key__"] = chunk.apply(make_composite_key, axis=1)
    mask_new_combo = ~chunk["__dedup_key__"].isin(seen_composite_keys)
    chunk = chunk[mask_new_combo].copy()
    seen_composite_keys.update(chunk["__dedup_key__"].tolist())
    chunk = chunk.drop(columns=["__dedup_key__"])

    return chunk


def run_full_clean():
    first = True

    for chunk in pd.read_csv(input_path, chunksize=chunksize):
        print("Processing new chunk...")
        cleaned_chunk = process_chunk(chunk, first)

        cleaned_chunk.to_csv(
            output_path,
            mode="w" if first else "a",
            header=first,
            index=False
        )
        first = False

    print("✅ Finished: full cleaned dataset written to:", output_path)


if __name__ == "__main__":
    run_full_clean()


Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new c

2- clean skill migration public dataset 

In [7]:
import pandas as pd

# 1. Load raw file
skills_raw = pd.read_csv("skill_migration_public.csv")
print(skills_raw.shape)
print(skills_raw.head())

(17617, 12)
  country_code country_name   wb_income   wb_region  skill_group_id  \
0           af  Afghanistan  Low income  South Asia            2549   
1           af  Afghanistan  Low income  South Asia            2608   
2           af  Afghanistan  Low income  South Asia            3806   
3           af  Afghanistan  Low income  South Asia           50321   
4           af  Afghanistan  Low income  South Asia            1606   

          skill_group_category        skill_group_name  net_per_10K_2015  \
0                  Tech Skills  Information Management           -791.59   
1              Business Skills  Operational Efficiency          -1610.25   
2  Specialized Industry Skills       National Security          -1731.45   
3                  Tech Skills        Software Testing           -957.50   
4  Specialized Industry Skills                    Navy          -1510.71   

   net_per_10K_2016  net_per_10K_2017  net_per_10K_2018  net_per_10K_2019  
0           -705.88         

In [8]:
# 2. Keep only the useful columns
skills = skills_raw[[
    "skill_group_name",
    "skill_group_category"
]].copy()

# 3. Drop rows where skill name is missing
skills = skills[skills["skill_group_name"].notna()].copy()

# 4. Normalize text (lowercase, strip)
for col in ["skill_group_name", "skill_group_category"]:
    if col in skills.columns:
        skills[col] = (
            skills[col]
            .astype(str)
            .str.strip()
            .str.lower()
        )

# 5. Drop exact duplicates on (skill_group_name, skill_group_category)
skills = skills.drop_duplicates(subset=["skill_group_name", "skill_group_category"])

# 6. If one skill appears with multiple categories, we keep the first one
skills = skills.drop_duplicates(subset=["skill_group_name"], keep="first")

print(skills.shape)
print(skills.head())


(249, 2)
         skill_group_name         skill_group_category
0  information management                  tech skills
1  operational efficiency              business skills
2       national security  specialized industry skills
3        software testing                  tech skills
4                    navy  specialized industry skills


In [9]:
skills.to_csv("skill_migration_clean.csv", index=False)
print("✅ Saved cleaned skills dictionary to skill_migration_clean.csv")

✅ Saved cleaned skills dictionary to skill_migration_clean.csv


3- Map skills onto all_jobs_clean_full.csv with chunks

Cell 1: Load skills dictionary

In [1]:
import pandas as pd
import re

skills_meta = pd.read_csv("skill_migration_clean.csv")

skill_dict = (
    skills_meta[["skill_group_name", "skill_group_category"]]
    .drop_duplicates()
    .set_index("skill_group_name")["skill_group_category"]
    .to_dict()
)

print(f"Loaded {len(skill_dict)} skills.")


Loaded 249 skills.


Cell 2: Define helper functions

In [2]:
def parse_skills(raw):
    if pd.isna(raw):
        return []
    text = str(raw).strip().lower()
    if not text:
        return []
    parts = re.split(r",|/|;|\||\+", text)
    return [p.strip() for p in parts if p.strip()]

def map_skill_categories(skill_list):
    cats = set()
    for s in skill_list:
        cat = skill_dict.get(s)
        if cat is None:
            cat = "other"
        cats.add(cat)
    if not cats:
        return ""
    return ",".join(sorted(cats))


Cell 3: Process all_jobs_clean_full.csv with chunks

In [3]:
input_path  = "all_jobs_clean_full.csv"
output_path = "all_jobs_mapped.csv"
chunksize   = 50_000

first = True

for chunk in pd.read_csv(input_path, chunksize=chunksize):
    print(f"Processing {len(chunk)} rows...")

    if "skills_raw" not in chunk.columns:
        chunk["skills_raw"] = ""

    chunk["skill_list"] = chunk["skills_raw"].apply(parse_skills)
    chunk["skill_categories"] = chunk["skill_list"].apply(map_skill_categories)

    chunk.to_csv(
        output_path,
        mode="w" if first else "a",
        header=first,
        index=False
    )
    first = False

print("✅ Done ! Final dataset:", output_path)


Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...


In [4]:
import os
print(os.path.getsize("all_jobs_mapped.csv") / (1024*1024), "MB")


6904.832558631897 MB


In [12]:
import pandas as pd
df = pd.read_csv("all_jobs_mapped.csv")
df.shape


C:\Users\Admin\AppData\Local\Temp\ipykernel_29532\1045503748.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("all_jobs_mapped.csv")


(2884556, 9)

In [9]:
df.columns

Index(['source', 'job_key', 'job_title', 'job_description', 'skills_raw',
       'location', 'company', 'skill_list', 'skill_categories'],
      dtype='object')

TRAINING STEPS TO FIND MODELS 

In [ ]:
import pandas as pd

path = "all_jobs_mapped.csv"

# Load a manageable sample for experiments (you can increase nrows later)
df = pd.read_csv(path, nrows=500_000)

df.shape, df.columns


--Ensure skill_list is really a lis

In [ ]:
import ast

def to_skill_list(x):
    if isinstance(x, list):
        return x
    if pd.isna(x):
        return []
    try:
        val = ast.literal_eval(x)
        if isinstance(val, list):
            return [str(s).strip().lower() for s in val]
    except:
        pass
    return [s.strip().lower() for s in str(x).split(",") if s.strip()]

df["skill_list"] = df["skill_list"].apply(to_skill_list)
df["skill_categories"] = df["skill_categories"].fillna("").astype(str)


Association Rules: 3 Models 